# OCR - IIIT5K dataset

## Installing dependencies

In [ ]:
!apt-get install tesseract-ocr libtesseract-dev tesseract-ocr-por
!apt-get install poppler-utils
!pip install pdf2image pytesseract
!pip install fitz
!pip install PyMuPDF
!pip install jiwer
!pip install textbsr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-por is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!pip install torch>=1.8.1 torchvision>=0.9 cnstd==1.2
!pip install numpy==1.23

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 88.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.0 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.0 which is incompatible.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.23.0 which is incompatible.


In [ ]:
!git clone https://github.com/JaidedAI/EasyOCR.git
!pip install easyocr
!pip install jiwer

Cloning into 'EasyOCR'...
remote: Enumerating objects: 2736, done.
remote: Total 2736 (delta 0), reused 0 (delta 0), pack-reused 2736
Receiving objects: 100% (2736/2736), 157.83 MiB | 14.69 MiB/s, done.
Resolving deltas: 100% (1664/1664), done.
Updating files: 100% (313/313), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 45.0 MB/s eta 0:00:00


## Load libraries


In [ ]:
import pytesseract
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

from pdf2image import convert_from_path, convert_from_bytes


## Downloading the dataset

In [ ]:
!wget https://cvit.iiit.ac.in/images/Projects/SceneTextUnderstanding/IIIT5K-Word_V3.0.tar.gz --no-check-certificate
!mkdir IIIT5K
!tar -xzvf /content/IIIT5K-Word_V3.0.tar.gz -C IIIT5K/

A saída de streaming foi truncada nas últimas 5000 linhas.
IIIT5K/test/1053_7.png
IIIT5K/test/1055_1.png
IIIT5K/test/1056_1.png
IIIT5K/test/1056_2.png
IIIT5K/test/1059_1.png
IIIT5K/test/1059_2.png
IIIT5K/test/1060_1.png
IIIT5K/test/1062_11.png
IIIT5K/test/1062_12.png
IIIT5K/test/1062_13.png
IIIT5K/test/1062_4.png
IIIT5K/test/1062_5.png
IIIT5K/test/1062_6.png
IIIT5K/test/1063_2.png
IIIT5K/test/1063_3.png
IIIT5K/test/1063_4.png
IIIT5K/test/1063_6.png
IIIT5K/test/1065_1.png
IIIT5K/test/1065_2.png
IIIT5K/test/1065_4.png
IIIT5K/test/1065_6.png
IIIT5K/test/106_1.png
IIIT5K/test/106_2.png
IIIT5K/test/106_3.png
IIIT5K/test/106_4.png
IIIT5K/test/106_5.png
IIIT5K/test/106_6.png
IIIT5K/test/1072_1.png
IIIT5K/test/1072_2.png
IIIT5K/test/1073_1.png
IIIT5K/test/1073_2.png
IIIT5K/test/107_1.png
IIIT5K/test/107_2.png
IIIT5K/test/107_3.png
IIIT5K/test/107_4.png
IIIT5K/test/1082_1.png
IIIT5K/test/1082_2.png
IIIT5K/test/108_1.png
IIIT5K/test/108_11.png
IIIT5K/test/108_13.png
IIIT5K/test/108_2.png
IIIT5K/

In [ ]:
import scipy.io

mat = scipy.io.loadmat('/content/IIIT5K/IIIT5K/testdata.mat')
data = mat

In [ ]:
IIIT5K_data = []

for i in range(0, len(data['testdata'][0])):
    filename = data['testdata'][0][i][0][0].split('/')[1]
    IIIT5K_data.append([filename, data['testdata'][0][i][1][0], '/content/IIIT5K/IIIT5K/test/', 'IIT5K'])


In [ ]:
IIIT5K_df = pd.DataFrame(IIIT5K_data)
IIIT5K_df.columns = ['filename',	'words',	'path',	'type']
IIIT5K_df

,filename,words,path,type
0,1002_1.png,PRIVATE,/content/IIIT5K/IIIT5K/test/,IIT5K
1,1002_2.png,PARKING,/content/IIIT5K/IIIT5K/test/,IIT5K
2,1009_1.png,SALUTES,/content/IIIT5K/IIIT5K/test/,IIT5K
3,100_1.png,DOLCE,/content/IIIT5K/IIIT5K/test/,IIT5K
4,100_2.png,GABBANA,/content/IIIT5K/IIIT5K/test/,IIT5K
...,...,...,...,...
2995,997_9.png,DAY,/content/IIIT5K/IIIT5K/test/,IIT5K
2996,999_1.png,HOW,/content/IIIT5K/IIIT5K/test/,IIT5K
2997,999_6.png,AT,/content/IIIT5K/IIIT5K/test/,IIT5K
2998,999_7.png,70,/content/IIIT5K/IIIT5K/test/,IIT5K


In [ ]:
for index, row in IIIT5K_df.iterrows():
  # Obtendo o arquivo da imagem
  path = row['path']  + row['filename']
  print(path)
  path = path.replace("jpg", "png")


/content/IIIT5K/IIIT5K/test/1002_1.png
/content/IIIT5K/IIIT5K/test/1002_2.png
/content/IIIT5K/IIIT5K/test/1009_1.png
/content/IIIT5K/IIIT5K/test/100_1.png
/content/IIIT5K/IIIT5K/test/100_2.png
/content/IIIT5K/IIIT5K/test/100_3.png
/content/IIIT5K/IIIT5K/test/1016_1.png
/content/IIIT5K/IIIT5K/test/1016_2.png
/content/IIIT5K/IIIT5K/test/1016_3.png
/content/IIIT5K/IIIT5K/test/1016_4.png
/content/IIIT5K/IIIT5K/test/1017_3.png
/content/IIIT5K/IIIT5K/test/101_1.png
/content/IIIT5K/IIIT5K/test/101_2.png
/content/IIIT5K/IIIT5K/test/101_3.png
/content/IIIT5K/IIIT5K/test/101_4.png
/content/IIIT5K/IIIT5K/test/101_5.png
/content/IIIT5K/IIIT5K/test/101_6.png
/content/IIIT5K/IIIT5K/test/1021_2.png
/content/IIIT5K/IIIT5K/test/1021_3.png
/content/IIIT5K/IIIT5K/test/1021_4.png
/content/IIIT5K/IIIT5K/test/1021_5.png
/content/IIIT5K/IIIT5K/test/1023_12.png
/content/IIIT5K/IIIT5K/test/1023_13.png
/content/IIIT5K/IIIT5K/test/1023_14.png
/content/IIIT5K/IIIT5K/test/1023_19.png
/content/IIIT5K/IIIT5K/test/10

In [ ]:
import time

def get_image_gray(image):
      inicio = time.time()
      roi_image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      fim = time.time()
      tempo_decorrido = fim - inicio

      return roi_image_gray, tempo_decorrido

def get_image_image_trans(image):
      inicio = time.time()
      ret, roi_image_trans = cv2.threshold(image, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
      fim = time.time()
      tempo_decorrido = fim - inicio

      return roi_image_trans, tempo_decorrido

def get_image_binary(image):
      inicio = time.time()
      _, roi_image_binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
      fim = time.time()
      tempo_decorrido = fim - inicio

      return roi_image_binary, tempo_decorrido

def get_image_trans_bilateral(image):
      inicio = time.time()
      roi_image_trans_bilateral = cv2.bilateralFilter(image, 9, 75, 75)
      fim = time.time()
      tempo_decorrido = fim - inicio

      return roi_image_trans_bilateral, tempo_decorrido


def get_pytesseract_image_to_string(image):
      inicio = time.time()
      text = pytesseract.image_to_string(image , lang='eng')
      fim = time.time()
      tempo_decorrido = fim - inicio

      return text, tempo_decorrido


def get_easyocr_readtext(reader, image):
      inicio = time.time()
      prediction = reader.readtext(image, detail = 0)
      fim = time.time()
      tempo_decorrido = fim - inicio

      return prediction, tempo_decorrido


In [ ]:
from PIL import Image
import easyocr
from jiwer import wer, cer, wil, wip, mer

def get_cv2_pytesseract_extraction(docs_test_df):
    dict_values = []

    for index, row in docs_test_df.iterrows():
      path = row['path']  + row['filename']
      print(path)
      path = path.replace("jpg", "png")

      image = cv2.imread(path)


      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(image)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")

      dict_values.append([row['path'], row['filename'], 'original', clean_text, None, tempo_decorrido_pytesseract])

      roi_image_gray, tempo_decorrido_image_gray = get_image_gray(image)
      roi_image_trans, tempo_decorrido_image_trans = get_image_image_trans(roi_image_gray)
      roi_image_binary, tempo_decorrido_image_binary = get_image_binary(roi_image_gray)
      roi_image_trans_bilateral, tempo_decorrido_image_trans_bilateral = get_image_trans_bilateral(roi_image_gray)

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_gray)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'image_full_gray', clean_text, tempo_decorrido_image_gray, tempo_decorrido_pytesseract])

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_trans)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'limiarização_de_otsu',  clean_text, tempo_decorrido_image_trans, tempo_decorrido_pytesseract])

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_binary)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'limiarização_binária ',  clean_text, tempo_decorrido_image_binary, tempo_decorrido_pytesseract])

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_trans_bilateral)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'bilateral',  clean_text, tempo_decorrido_image_trans_bilateral, tempo_decorrido_pytesseract])

    return dict_values

def get_cv2_easyocr_extraction(docs_test_df):
    reader = easyocr.Reader(['en'])

    dict_values = []
    for index, row in docs_test_df.iterrows():

      path = row['path']  + row['filename']
      print(path)
      path = path.replace("jpg", "png")
      image = cv2.imread(path)

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, image)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'original', prediction, None, tempo_decorrido_easyocr])

      roi_image_gray, tempo_decorrido_image_gray = get_image_gray(image)
      roi_image_trans, tempo_decorrido_image_trans = get_image_image_trans(roi_image_gray)
      roi_image_binary, tempo_decorrido_image_binary = get_image_binary(roi_image_gray)
      roi_image_trans_bilateral, tempo_decorrido_image_trans_bilateral = get_image_trans_bilateral(roi_image_gray)

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_gray)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'image_full_gray', prediction, tempo_decorrido_image_gray, tempo_decorrido_easyocr])

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_trans)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'limiarização_de_otsu', prediction, tempo_decorrido_image_trans, tempo_decorrido_easyocr])

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_binary)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'limiarização_binária', prediction, tempo_decorrido_image_binary, tempo_decorrido_easyocr])

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_trans_bilateral)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'bilateral', prediction, tempo_decorrido_image_trans_bilateral, tempo_decorrido_easyocr])

    return dict_values


def get_esrgan_pytesseract_extraction(docs_test_df):
    dict_values = []
    path_folder = '/content/crop_craft-images/improv_test_images/'

    for index, row in docs_test_df.iterrows():

      path = path_folder
      path = path + row['filename']
      path = path.replace("jpg", "png")
      print(path)

      image = cv2.imread(path)

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(image)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")

      dict_values.append([row['path'], row['filename'], 'original', clean_text, None, tempo_decorrido_pytesseract])

      roi_image_gray, tempo_decorrido_image_gray = get_image_gray(image)
      roi_image_trans, tempo_decorrido_image_trans = get_image_image_trans(roi_image_gray)
      roi_image_binary, tempo_decorrido_image_binary = get_image_binary(roi_image_gray)
      roi_image_trans_bilateral, tempo_decorrido_image_trans_bilateral = get_image_trans_bilateral(roi_image_gray)

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_gray)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'image_full_gray',  clean_text, tempo_decorrido_image_gray, tempo_decorrido_pytesseract])

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_trans)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'limiarização_de_otsu', clean_text, tempo_decorrido_image_trans, tempo_decorrido_pytesseract])

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_binary)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'limiarização_binária ', clean_text, tempo_decorrido_image_binary, tempo_decorrido_pytesseract])

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_trans_bilateral)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'bilateral', clean_text, tempo_decorrido_image_trans_bilateral, tempo_decorrido_pytesseract])

    return dict_values


def get_esrgan_easyocr_extraction(docs_test_df):
    reader = easyocr.Reader(['pt'])

    dict_values = []
    path_folder = '/content/crop_craft-images/improv_test_images/'

    for index, row in docs_test_df.iterrows():

      path = path_folder

      path = path + row['filename']
      path = path.replace("jpg", "png")
      print(path)

      image = cv2.imread(path)

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, image)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'original', prediction, None, tempo_decorrido_easyocr])

      roi_image_gray, tempo_decorrido_image_gray = get_image_gray(image)
      roi_image_trans, tempo_decorrido_image_trans = get_image_image_trans(roi_image_gray)
      roi_image_binary, tempo_decorrido_image_binary = get_image_binary(roi_image_gray)
      roi_image_trans_bilateral, tempo_decorrido_image_trans_bilateral = get_image_trans_bilateral(roi_image_gray)

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_gray)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'image_full_gray',  prediction, tempo_decorrido_image_gray, tempo_decorrido_easyocr])

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_trans)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'limiarização_de_otsu', prediction, tempo_decorrido_image_trans, tempo_decorrido_easyocr])

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_binary)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'limiarização_binária', prediction, tempo_decorrido_image_binary, tempo_decorrido_easyocr])

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_trans_bilateral)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'bilateral', prediction, tempo_decorrido_image_trans_bilateral, tempo_decorrido_easyocr])

    return dict_values

def process_result(dict_values):

  data_df = pd.DataFrame(dict_values)
  data_df.columns = ['path', 'filename', 'filter',  'prediction', 'cv2 time', 'prediction time']

  data_df['words'] = ''

  for index, row in data_df.iterrows():
    data = IIIT5K_df[IIIT5K_df['filename'] == row['filename']]['words']
    if data.values:
      data_df.at[index, 'words'] = data.values[0]

  return data_df


def get_metrics(data_df):

  for index, row in data_df.iterrows():
    data_df.at[index, 'wer'] = wer(row['words'], ''.join(row['prediction']) )
    data_df.at[index, 'cer'] = cer(row['words'], ''.join(row['prediction']) )
    data_df.at[index, 'wil'] = wil(row['words'], ''.join(row['prediction']) )
    data_df.at[index, 'wip'] = wip(row['words'], ''.join(row['prediction']) )
    data_df.at[index, 'mer'] = mer(row['words'], ''.join(row['prediction']) )

  return data_df

### OpenCV[cinza] -> Tesseract | fluxo 4: CRAFT -> OpenCV[todos preproc] -> Tesseract

In [ ]:
dict_values = get_cv2_pytesseract_extraction(IIIT5K_df)

/content/IIIT5K/IIIT5K/test/1002_1.png
/content/IIIT5K/IIIT5K/test/1002_2.png
/content/IIIT5K/IIIT5K/test/1009_1.png
/content/IIIT5K/IIIT5K/test/100_1.png
/content/IIIT5K/IIIT5K/test/100_2.png
/content/IIIT5K/IIIT5K/test/100_3.png
/content/IIIT5K/IIIT5K/test/1016_1.png
/content/IIIT5K/IIIT5K/test/1016_2.png
/content/IIIT5K/IIIT5K/test/1016_3.png
/content/IIIT5K/IIIT5K/test/1016_4.png
/content/IIIT5K/IIIT5K/test/1017_3.png
/content/IIIT5K/IIIT5K/test/101_1.png
/content/IIIT5K/IIIT5K/test/101_2.png
/content/IIIT5K/IIIT5K/test/101_3.png
/content/IIIT5K/IIIT5K/test/101_4.png
/content/IIIT5K/IIIT5K/test/101_5.png
/content/IIIT5K/IIIT5K/test/101_6.png
/content/IIIT5K/IIIT5K/test/1021_2.png
/content/IIIT5K/IIIT5K/test/1021_3.png
/content/IIIT5K/IIIT5K/test/1021_4.png
/content/IIIT5K/IIIT5K/test/1021_5.png
/content/IIIT5K/IIIT5K/test/1023_12.png
/content/IIIT5K/IIIT5K/test/1023_13.png
/content/IIIT5K/IIIT5K/test/1023_14.png
/content/IIIT5K/IIIT5K/test/1023_19.png
/content/IIIT5K/IIIT5K/test/10

In [ ]:
data_df = process_result(dict_values)

In [ ]:
data_df = get_metrics(data_df)

In [ ]:
approach_3_4_data_df = data_df
approach_3_4_data_df['fluxo'] = 'OpenCV[todos preproc] -> Tesseract'

approach_3_4_data_df.head(20)

,path,filename,filter,prediction,cv2 time,prediction time,words,wer,cer,wil,wip,mer,fluxo
0,/content/IIIT5K/IIIT5K/test/,1002_1.png,original,,NaN,0.120728,PRIVATE,1.0,1.000000,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
1,/content/IIIT5K/IIIT5K/test/,1002_1.png,image_full_gray,,0.000159,0.107295,PRIVATE,1.0,1.000000,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
2,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_de_otsu,,0.000069,0.107000,PRIVATE,1.0,1.000000,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
3,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_binária,,0.000005,0.107108,PRIVATE,1.0,1.000000,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
4,/content/IIIT5K/IIIT5K/test/,1002_1.png,bilateral,,0.010988,0.106032,PRIVATE,1.0,1.000000,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
5,/content/IIIT5K/IIIT5K/test/,1002_2.png,original,,NaN,0.110778,PARKING,1.0,1.000000,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
6,/content/IIIT5K/IIIT5K/test/,1002_2.png,image_full_gray,,0.000039,0.105239,PARKING,1.0,1.000000,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
7,/content/IIIT5K/IIIT5K/test/,1002_2.png,limiarização_de_otsu,,0.000023,0.104982,PARKING,1.0,1.000000,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
8,/content/IIIT5K/IIIT5K/test/,1002_2.png,limiarização_binária,,0.000005,0.105244,PARKING,1.0,1.000000,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
9,/content/IIIT5K/IIIT5K/test/,1002_2.png,bilateral,,0.001026,0.104709,PARKING,1.0,1.000000,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract


### OpenCV[cinza] -> EASYOCR | fluxo 2: CRAFT -> OpenCV[todos preproc] -> EASYOCR

In [ ]:
dict_values = get_cv2_easyocr_extraction(IIIT5K_df)

/content/IIIT5K/IIIT5K/test/1002_1.png
/content/IIIT5K/IIIT5K/test/1002_2.png
/content/IIIT5K/IIIT5K/test/1009_1.png
/content/IIIT5K/IIIT5K/test/100_1.png
/content/IIIT5K/IIIT5K/test/100_2.png
/content/IIIT5K/IIIT5K/test/100_3.png
/content/IIIT5K/IIIT5K/test/1016_1.png
/content/IIIT5K/IIIT5K/test/1016_2.png
/content/IIIT5K/IIIT5K/test/1016_3.png
/content/IIIT5K/IIIT5K/test/1016_4.png
/content/IIIT5K/IIIT5K/test/1017_3.png
/content/IIIT5K/IIIT5K/test/101_1.png
/content/IIIT5K/IIIT5K/test/101_2.png
/content/IIIT5K/IIIT5K/test/101_3.png
/content/IIIT5K/IIIT5K/test/101_4.png
/content/IIIT5K/IIIT5K/test/101_5.png
/content/IIIT5K/IIIT5K/test/101_6.png
/content/IIIT5K/IIIT5K/test/1021_2.png
/content/IIIT5K/IIIT5K/test/1021_3.png
/content/IIIT5K/IIIT5K/test/1021_4.png
/content/IIIT5K/IIIT5K/test/1021_5.png
/content/IIIT5K/IIIT5K/test/1023_12.png
/content/IIIT5K/IIIT5K/test/1023_13.png
/content/IIIT5K/IIIT5K/test/1023_14.png
/content/IIIT5K/IIIT5K/test/1023_19.png
/content/IIIT5K/IIIT5K/test/10

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


/content/IIIT5K/IIIT5K/test/5114_1.png
/content/IIIT5K/IIIT5K/test/5114_3.png
/content/IIIT5K/IIIT5K/test/5114_4.png
/content/IIIT5K/IIIT5K/test/5114_5.png
/content/IIIT5K/IIIT5K/test/5115_3.png
/content/IIIT5K/IIIT5K/test/5116_2.png
/content/IIIT5K/IIIT5K/test/5116_3.png
/content/IIIT5K/IIIT5K/test/5116_4.png
/content/IIIT5K/IIIT5K/test/5118_1.png
/content/IIIT5K/IIIT5K/test/5118_2.png
/content/IIIT5K/IIIT5K/test/5118_4.png
/content/IIIT5K/IIIT5K/test/5119_1.png
/content/IIIT5K/IIIT5K/test/5119_2.png
/content/IIIT5K/IIIT5K/test/5119_3.png
/content/IIIT5K/IIIT5K/test/511_1.png
/content/IIIT5K/IIIT5K/test/511_6.png
/content/IIIT5K/IIIT5K/test/511_7.png
/content/IIIT5K/IIIT5K/test/511_8.png
/content/IIIT5K/IIIT5K/test/5129_10.png
/content/IIIT5K/IIIT5K/test/5129_4.png
/content/IIIT5K/IIIT5K/test/5129_5.png
/content/IIIT5K/IIIT5K/test/5129_6.png
/content/IIIT5K/IIIT5K/test/5129_7.png
/content/IIIT5K/IIIT5K/test/5130_1.png
/content/IIIT5K/IIIT5K/test/5130_2.png
/content/IIIT5K/IIIT5K/test/

In [ ]:
data_df = process_result(dict_values)

In [ ]:
data_df = get_metrics(data_df)

In [ ]:
approach_1_2_data_df = data_df.copy()
approach_1_2_data_df['fluxo'] = 'OpenCV[todos preproc] -> EASYOCR'

approach_1_2_data_df.head()

,path,filename,filter,prediction,cv2 time,prediction time,words,wer,cer,wil,wip,mer,fluxo
0,/content/IIIT5K/IIIT5K/test/,1002_1.png,original,PRIVATE,NaN,0.122835,PRIVATE,0.0,0.0,0.0,1.0,0.0,OpenCV[todos preproc] -> EASYOCR
1,/content/IIIT5K/IIIT5K/test/,1002_1.png,image_full_gray,PRIVATE,0.000030,0.014177,PRIVATE,0.0,0.0,0.0,1.0,0.0,OpenCV[todos preproc] -> EASYOCR
2,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_de_otsu,PRIVATE,0.000044,0.013822,PRIVATE,0.0,0.0,0.0,1.0,0.0,OpenCV[todos preproc] -> EASYOCR
3,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_binária,PRIVATE,0.000005,0.013535,PRIVATE,0.0,0.0,0.0,1.0,0.0,OpenCV[todos preproc] -> EASYOCR
4,/content/IIIT5K/IIIT5K/test/,1002_1.png,bilateral,PRIVATE,0.001289,0.013451,PRIVATE,0.0,0.0,0.0,1.0,0.0,OpenCV[todos preproc] -> EASYOCR


In [ ]:
result = pd.concat([approach_1_2_data_df, approach_3_4_data_df])
result

,path,filename,filter,prediction,cv2 time,prediction time,words,wer,cer,wil,wip,mer,fluxo
0,/content/IIIT5K/IIIT5K/test/,1002_1.png,original,PRIVATE,NaN,0.122835,PRIVATE,0.0,0.0,0.0,1.0,0.0,OpenCV[todos preproc] -> EASYOCR
1,/content/IIIT5K/IIIT5K/test/,1002_1.png,image_full_gray,PRIVATE,0.000030,0.014177,PRIVATE,0.0,0.0,0.0,1.0,0.0,OpenCV[todos preproc] -> EASYOCR
2,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_de_otsu,PRIVATE,0.000044,0.013822,PRIVATE,0.0,0.0,0.0,1.0,0.0,OpenCV[todos preproc] -> EASYOCR
3,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_binária,PRIVATE,0.000005,0.013535,PRIVATE,0.0,0.0,0.0,1.0,0.0,OpenCV[todos preproc] -> EASYOCR
4,/content/IIIT5K/IIIT5K/test/,1002_1.png,bilateral,PRIVATE,0.001289,0.013451,PRIVATE,0.0,0.0,0.0,1.0,0.0,OpenCV[todos preproc] -> EASYOCR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,/content/IIIT5K/IIIT5K/test/,999_8.png,original,,NaN,0.103999,MPH,1.0,1.0,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
14996,/content/IIIT5K/IIIT5K/test/,999_8.png,image_full_gray,,0.000033,0.106816,MPH,1.0,1.0,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
14997,/content/IIIT5K/IIIT5K/test/,999_8.png,limiarização_de_otsu,,0.000020,0.104325,MPH,1.0,1.0,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract
14998,/content/IIIT5K/IIIT5K/test/,999_8.png,limiarização_binária,,0.000004,0.104883,MPH,1.0,1.0,1.0,0.0,1.0,OpenCV[todos preproc] -> Tesseract


## ESRGAN Model

In [ ]:
import os
import shutil
from textbsr import textbsr

Creating folder that will contain the enhanced images

In [ ]:
! mkdir /content/improv_test_images

In [ ]:
diretorio_destino = '/content/crop_craft-images/test_images'

textbsr.bsr(input_path='/content/IIIT5K/IIIT5K/test/', output_path='/content/improv_test_images', aligned=True)

print("Imagens copiadas com sucesso!")

               Model Name : BSRGAN
                   GPU ID : 0
               Input Path : /content/IIIT5K/IIIT5K/test/
              Output Path : /content/improv_test_images
       Background SR Path : None
            Image Details : Aligned Text Layout. No text detection is used.
 Save LR & SR text layout : False
          Restoring  0001 : x4 --> 1002_1.png
          Restoring  0002 : x4 --> 1002_2.png
          Restoring  0003 : x4 --> 1009_1.png
          Restoring  0004 : x4 --> 100_1.png
          Restoring  0005 : x4 --> 100_2.png
          Restoring  0006 : x4 --> 100_3.png
          Restoring  0007 : x4 --> 1016_1.png
          Restoring  0008 : x4 --> 1016_2.png
          Restoring  0009 : x4 --> 1016_3.png
          Restoring  0010 : x4 --> 1016_4.png
          Restoring  0011 : x4 --> 1017_3.png
          Restoring  0012 : x4 --> 101_1.png
          Restoring  0013 : x4 --> 101_2.png
          Restoring  0014 : x4 --> 101_3.png
          Restoring  0015 : x4 --> 101_4.

In [ ]:
pasta = "/content/improv_test_images/"  # Substitua pelo caminho da pasta que você deseja listar

arquivos = [f for f in os.listdir(pasta) if os.path.isfile(os.path.join(pasta, f)) ]
arquivos.sort()

In [ ]:
nova_lista = [nome.replace('_BSRGANText', '') for nome in arquivos]
pasta = '/content/improv_test_images'

for arquivo_original, novo_nome in zip(arquivos, nova_lista):
    caminho_original = os.path.join(pasta, arquivo_original)
    caminho_novo = os.path.join(pasta, novo_nome)

    # Renomeia o arquivo
    os.rename(caminho_original, caminho_novo)

print("Arquivos renomeados com sucesso!")

Arquivos renomeados com sucesso!


### 'ESRGAN -> OpenCV -> Tesseract'

In [ ]:
def get_esrgan_pytesseract_extraction(docs_test_df):
    dict_values = []
    path_folder = '/content/improv_test_images/'

    for index, row in docs_test_df.iterrows():
      path = path_folder

      path = path + row['filename']
      path = path.replace("jpg", "png")
      print(path)
      image = cv2.imread(path)

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(image)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'original', clean_text, None, tempo_decorrido_pytesseract])

      roi_image_gray, tempo_decorrido_image_gray = get_image_gray(image)
      roi_image_trans, tempo_decorrido_image_trans = get_image_image_trans(roi_image_gray)
      roi_image_binary, tempo_decorrido_image_binary = get_image_binary(roi_image_gray)
      roi_image_trans_bilateral, tempo_decorrido_image_trans_bilateral = get_image_trans_bilateral(roi_image_gray)

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_gray)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'image_full_gray', clean_text, tempo_decorrido_image_gray, tempo_decorrido_pytesseract])

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_gray)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'limiarização_de_otsu', clean_text, tempo_decorrido_image_trans, tempo_decorrido_pytesseract])

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_binary)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'limiarização_binária ', clean_text, tempo_decorrido_image_binary, tempo_decorrido_pytesseract])

      text, tempo_decorrido_pytesseract = get_pytesseract_image_to_string(roi_image_trans_bilateral)
      clean_text = text.strip().replace("\n", " ").replace("\r", "")
      dict_values.append([row['path'], row['filename'], 'bilateral', clean_text, tempo_decorrido_image_trans_bilateral, tempo_decorrido_pytesseract])

    return dict_values

In [ ]:
dict_values = get_esrgan_pytesseract_extraction(IIIT5K_df)

/content/improv_test_images/1002_1.png
/content/improv_test_images/1002_2.png
/content/improv_test_images/1009_1.png
/content/improv_test_images/100_1.png
/content/improv_test_images/100_2.png
/content/improv_test_images/100_3.png
/content/improv_test_images/1016_1.png
/content/improv_test_images/1016_2.png
/content/improv_test_images/1016_3.png
/content/improv_test_images/1016_4.png
/content/improv_test_images/1017_3.png
/content/improv_test_images/101_1.png
/content/improv_test_images/101_2.png
/content/improv_test_images/101_3.png
/content/improv_test_images/101_4.png
/content/improv_test_images/101_5.png
/content/improv_test_images/101_6.png
/content/improv_test_images/1021_2.png
/content/improv_test_images/1021_3.png
/content/improv_test_images/1021_4.png
/content/improv_test_images/1021_5.png
/content/improv_test_images/1023_12.png
/content/improv_test_images/1023_13.png
/content/improv_test_images/1023_14.png
/content/improv_test_images/1023_19.png
/content/improv_test_images/10

In [ ]:
data_df = process_result(dict_values)

In [ ]:
data_df = get_metrics(data_df)

In [ ]:
approach_7_8_data_df = data_df.copy()
approach_7_8_data_df['fluxo'] =  'ESRGAN -> OpenCV -> Tesseract'

approach_7_8_data_df.head()

,path,filename,filter,prediction,cv2 time,prediction time,words,wer,cer,wil,wip,mer,fluxo
0,/content/IIIT5K/IIIT5K/test/,1002_1.png,original,PRIVATE,NaN,0.260870,PRIVATE,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> Tesseract
1,/content/IIIT5K/IIIT5K/test/,1002_1.png,image_full_gray,PRIVATE,0.000318,0.172107,PRIVATE,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> Tesseract
2,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_de_otsu,PRIVATE,0.000360,0.168947,PRIVATE,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> Tesseract
3,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_binária,PRIVATE,0.000024,0.133774,PRIVATE,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> Tesseract
4,/content/IIIT5K/IIIT5K/test/,1002_1.png,bilateral,AAA E,0.022127,0.163586,PRIVATE,2.0,0.714286,1.0,0.0,1.0,ESRGAN -> OpenCV -> Tesseract


###  'ESRGAN -> OpenCV -> EASYOCR'

In [ ]:
def get_esrgan_easyocr_extraction(docs_test_df):
    reader = easyocr.Reader(['en'])

    dict_values = []
    path_folder = '/content/improv_test_images/'

    for index, row in docs_test_df.iterrows():
      path = path_folder

      path = path + row['filename']
      path = path.replace("jpg", "png")
      print(path)
      image = cv2.imread(path)

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, image)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'original', prediction, None, tempo_decorrido_easyocr])

      roi_image_gray, tempo_decorrido_image_gray = get_image_gray(image)
      roi_image_trans, tempo_decorrido_image_trans = get_image_image_trans(roi_image_gray)
      roi_image_binary, tempo_decorrido_image_binary = get_image_binary(roi_image_gray)
      roi_image_trans_bilateral, tempo_decorrido_image_trans_bilateral = get_image_trans_bilateral(roi_image_gray)


      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_gray)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'image_full_gray', prediction, tempo_decorrido_image_gray, tempo_decorrido_easyocr])

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_trans)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'limiarização_de_otsu', prediction, tempo_decorrido_image_trans, tempo_decorrido_easyocr])

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_binary)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'limiarização_binária', prediction, tempo_decorrido_image_binary, tempo_decorrido_easyocr])

      prediction, tempo_decorrido_easyocr = get_easyocr_readtext(reader, roi_image_trans_bilateral)
      prediction = reader.readtext(roi_image_trans_bilateral, detail = 0)
      prediction = prediction[0] if len(prediction) > 0 else ''
      dict_values.append([row['path'], row['filename'], 'bilateral', prediction, tempo_decorrido_image_trans_bilateral, tempo_decorrido_easyocr])

    return dict_values


In [ ]:
dict_values = get_esrgan_easyocr_extraction(IIIT5K_df)

/content/improv_test_images/1002_1.png
/content/improv_test_images/1002_2.png
/content/improv_test_images/1009_1.png
/content/improv_test_images/100_1.png
/content/improv_test_images/100_2.png
/content/improv_test_images/100_3.png
/content/improv_test_images/1016_1.png
/content/improv_test_images/1016_2.png
/content/improv_test_images/1016_3.png
/content/improv_test_images/1016_4.png
/content/improv_test_images/1017_3.png
/content/improv_test_images/101_1.png
/content/improv_test_images/101_2.png
/content/improv_test_images/101_3.png
/content/improv_test_images/101_4.png
/content/improv_test_images/101_5.png
/content/improv_test_images/101_6.png
/content/improv_test_images/1021_2.png
/content/improv_test_images/1021_3.png
/content/improv_test_images/1021_4.png
/content/improv_test_images/1021_5.png
/content/improv_test_images/1023_12.png
/content/improv_test_images/1023_13.png
/content/improv_test_images/1023_14.png
/content/improv_test_images/1023_19.png
/content/improv_test_images/10

In [ ]:
data_df = process_result(dict_values)

In [ ]:
data_df = get_metrics(data_df)

In [ ]:
approach_5_6_data_df = data_df.copy()
approach_5_6_data_df['fluxo'] = 'ESRGAN -> OpenCV -> EASYOCR'

approach_5_6_data_df.head(20)

,path,filename,filter,prediction,cv2 time,prediction time,words,wer,cer,wil,wip,mer,fluxo
0,/content/IIIT5K/IIIT5K/test/,1002_1.png,original,PRIVATE,NaN,0.104903,PRIVATE,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
1,/content/IIIT5K/IIIT5K/test/,1002_1.png,image_full_gray,PRIVATE,0.000178,0.036666,PRIVATE,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
2,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_de_otsu,PRIVATE,0.000380,0.036533,PRIVATE,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
3,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_binária,PRIVATE,0.000019,0.035235,PRIVATE,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
4,/content/IIIT5K/IIIT5K/test/,1002_1.png,bilateral,PRIVATE,0.021881,0.035664,PRIVATE,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
5,/content/IIIT5K/IIIT5K/test/,1002_2.png,original,PARKING,NaN,0.097766,PARKING,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
6,/content/IIIT5K/IIIT5K/test/,1002_2.png,image_full_gray,PARKING,0.000165,0.041541,PARKING,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
7,/content/IIIT5K/IIIT5K/test/,1002_2.png,limiarização_de_otsu,PARKING,0.000392,0.038523,PARKING,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
8,/content/IIIT5K/IIIT5K/test/,1002_2.png,limiarização_binária,PARKING,0.000024,0.037944,PARKING,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
9,/content/IIIT5K/IIIT5K/test/,1002_2.png,bilateral,PARKING,0.027050,0.038641,PARKING,0.0,0.000000,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR


In [ ]:
result = pd.concat([approach_5_6_data_df, approach_7_8_data_df])
result

,path,filename,filter,prediction,cv2 time,prediction time,words,wer,cer,wil,wip,mer,fluxo
0,/content/IIIT5K/IIIT5K/test/,1002_1.png,original,PRIVATE,NaN,0.104903,PRIVATE,0.0,0.0,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
1,/content/IIIT5K/IIIT5K/test/,1002_1.png,image_full_gray,PRIVATE,0.000178,0.036666,PRIVATE,0.0,0.0,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
2,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_de_otsu,PRIVATE,0.000380,0.036533,PRIVATE,0.0,0.0,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
3,/content/IIIT5K/IIIT5K/test/,1002_1.png,limiarização_binária,PRIVATE,0.000019,0.035235,PRIVATE,0.0,0.0,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
4,/content/IIIT5K/IIIT5K/test/,1002_1.png,bilateral,PRIVATE,0.021881,0.035664,PRIVATE,0.0,0.0,0.0,1.0,0.0,ESRGAN -> OpenCV -> EASYOCR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,/content/IIIT5K/IIIT5K/test/,999_8.png,original,MPH,NaN,0.153610,MPH,0.0,0.0,0.0,1.0,0.0,ESRGAN -> OpenCV -> Tesseract
14996,/content/IIIT5K/IIIT5K/test/,999_8.png,image_full_gray,MPH,0.000155,0.127275,MPH,0.0,0.0,0.0,1.0,0.0,ESRGAN -> OpenCV -> Tesseract
14997,/content/IIIT5K/IIIT5K/test/,999_8.png,limiarização_de_otsu,MPH,0.000245,0.125907,MPH,0.0,0.0,0.0,1.0,0.0,ESRGAN -> OpenCV -> Tesseract
14998,/content/IIIT5K/IIIT5K/test/,999_8.png,limiarização_binária,MPH,0.000018,0.119319,MPH,0.0,0.0,0.0,1.0,0.0,ESRGAN -> OpenCV -> Tesseract
